# R2IFE Database App
Application de visualisation de la base de données des profils des membres du r2ife.

*   Bibliothèque utilisée : streamlit
*   Fichier source : Fichier_test.xlsx
*   L'affichage : en local

1 - Installation des differentes bibliothèques

❗Le code ipv récupéré est **à conserver**, pour l'affichage en local de l'application, plus de détails en partie 3.



In [1]:
# Installer de nvm (Node Version Manager)
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.1/install.sh | bash

# Sourcer le fichier de configuration de nvm pour rendre la commande nvm disponible
!source ~/.nvm/nvm.sh

# Installer Node.js v14.16.0
!nvm install v14.16.0

# Activer la version installée
!nvm use v14.16.0

!pip install streamlit -q
!pip install pyngrok --upgrade -q

# installer le tunel ouvrir le site en local
!npm install -g localtunnel -q
!wget -q -O - ipv4.icanhazip.com

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15037  100 15037    0     0  33645      0 --:--:-- --:--:-- --:--:-- 33639
=> Downloading nvm from git to '/root/.nvm'
=> Cloning into '/root/.nvm'...
remote: Enumerating objects: 365, done.
remote: Counting objects: 100% (365/365), done.
remote: Compressing objects: 100% (314/314), done.
remote: Total 365 (delta 43), reused 162 (delta 25), pack-reused 0
Receiving objects: 100% (365/365), 365.11 KiB | 1.63 MiB/s, done.
Resolving deltas: 100% (43/43), done.
* (HEAD detached at FETCH_HEAD)
  master
=> Compressing and cleaning up git repository

=> Appending nvm source string to /root/.bashrc
=> Appending bash_completion source string to /root/.bashrc
=> Close and reopen your terminal to start using nvm or run the following to use it now:

export NVM_DIR="$HOME/.nvm"
[ -s "$NVM_DIR/nvm.sh" ] && \. "$NVM_DIR/nvm.sh"  # This loads 

2 - Code source de l'application

*writefile app.py* permet ici d'écrire directement le code dans un fichier *app.py*. L'extension *.py* est effectivement necessaire au lancement de l'application streamlit

In [2]:
%%writefile app.py

# Importer les bibliothèques nécessaires
import streamlit as st
import pandas as pd
import base64
import urllib.request
import shutil
import os
import zipfile
# ####################### IMPORTATION DES FICHIERS ####################################################################

# Chargement du fichier Excel depuis Google Drive (via le lien de partage)
google_drive_link = "https://docs.google.com/spreadsheets/d/1GgMH4_rNkMNAl-4FBpmbgWwvGVbE_1p2/edit?usp=drive_link"

# Extraction de l'ID du fichier depuis le lien
file_id = google_drive_link.split('/')[-2]

# Construction du lien de téléchargement direct
download_link = f'https://drive.google.com/uc?id={file_id}'

excel_file = pd.ExcelFile(download_link)

# Lire le fichier Excel avec l'option sheet_name pour spécifier la feuille
df_raw = pd.read_excel(excel_file)

# Création du DataFrame
df_raw = pd.DataFrame(df_raw)
df = df_raw.copy()


column_order = ["Nom", "Prenoms", "Niveau", "Formation", "Etablissement","Email", "Telephone", "Competences","CV"]
df = df[column_order]

# ########################## ENTETE #######################################################################################

# Ajout du logo, position au niveau de barre à gauche
logo_url = f'https://drive.google.com/file/d/1KHwPzRS3ezd483wNta5KIvhQGZULrC1s/view?usp=drive_link'
st.image("LOGOS (2).png", width=100, output_format = "PNG")

st.title(":orange[Base de données pour recrutement]")
st.write(
  """

  """
)
st.write("""Le Réseau des Ingénieurs Ivoiriens Formés à l'Etranger regroupe un grand nombre de talents de la diaspora qui apporteront beaucoup à vos entreprises. Grâce à cet outil, vous pouvez accéder aux différents CVs et prendre contact avec les profils qui vous intéressent.""" )

st.write(
  """

  """
)
st.markdown('### :pushpin: Données sur les profils Ingénieurs')


############ Ajout Noellie  FILTRES #########################################################################
# Liste des modalités de la colonne à filtrer
all_formations = df['Formation'].unique()
all_etablissement = df['Etablissement'].unique()
all_niveau = df['Niveau'].unique()

# Sélection de la modalité via une liste déroulante

col1,col2,col3 = st.columns(3)

with col1:
    selected_formation = st.selectbox("Domaine de formation", [""] + list(all_formations), index=None, placeholder="Choisir une option")

with col2:
    selected_etablissement = st.selectbox("Etablissement", [""] + list(all_etablissement), index=None, placeholder="Choisir une option")

with col3:
    selected_niveau = st.selectbox("Niveau d'étude", [""] + list(all_niveau), index=None, placeholder="Choisir une option")


if selected_formation or selected_etablissement or selected_niveau:
    condition_formation = df['Formation'] == selected_formation if selected_formation else True
    condition_etablissement = df['Etablissement'] == selected_etablissement if selected_etablissement else True
    condition_niveau = df['Niveau'] == selected_niveau if selected_niveau else True
    df_filtre = df[condition_formation & condition_etablissement & condition_niveau ]
    st.write("Données filtrées :")
    st.data_editor(
        df_filtre,
        column_config={"CV": st.column_config.LinkColumn("Lien vers le CV",display_text="Ouvrir le lien",disabled = True),
        "Nom": st.column_config.Column(disabled = True),
        "Prenoms": st.column_config.Column(disabled = True),
        "Niveau": st.column_config.Column(disabled = True),
        "Formation": st.column_config.Column(disabled = True),
        "Etablissement": st.column_config.Column(disabled = True),
        "Email": st.column_config.Column(disabled = True),
        "Telephone": st.column_config.Column(disabled = True),
        "Competences": st.column_config.Column(disabled = True)},hide_index=True)
else:
    st.data_editor(
        df,
        column_config={"CV": st.column_config.LinkColumn("Lien vers le CV",display_text="Ouvrir le lien",disabled = True),
        "Nom": st.column_config.Column(disabled = True),
        "Prenoms": st.column_config.Column(disabled = True),
        "Niveau": st.column_config.Column(disabled = True),
        "Formation": st.column_config.Column(disabled = True),
        "Etablissement": st.column_config.Column(disabled = True),
        "Email": st.column_config.Column(disabled = True),
        "Telephone": st.column_config.Column(disabled = True),
        "Competences": st.column_config.Column(disabled = True)},hide_index=True)

st.markdown("_Dernière mise à jour : Novembre 2023_")


st.write(
  """



  """
)

st.sidebar.write(
  """
  Plus d'infos sur nous:

  :link: www.r2ife-inge.com"""
)

st.sidebar.write(
  """Contacts :

  :e-mail: r2ife.reseau@gmail.com

  :telephone_receiver: +33 767389260 (Murielle Yao)

  :telephone_receiver: +225 09200191 (Kemi Ilupeju)"""
)



Writing app.py


3 - Lancement de l'application Streamlit

*localtunnel* permet de lancer l'application en local. 3 liens vont s'afficher, cliquer sur le lien contenant "loca.lit" et entrer le code de la partie 1.

In [ ]:
# !streamlit run --browser.gatherUsageStats=False app.py
# !streamlit run --browser.gatherUsageStats=False --server.port 8501 app.py

!streamlit run app.py & npx localtunnel --port 8501



npx: installed 22 in 2.81s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.155.210.201:8501

your url is: https://tall-carrots-cross.loca.lt


In [ ]:
# DRAFT
# Affichez chaque ligne avec un bouton pour le lien
for index, row in filtered_df.iterrows():
    st.write(f"{row['Nom']} {row['Prenoms']}")
    link_button = st.button(f"Ouvrir fichier {index + 1}")
    if link_button:
        st.write(f"Ouvrir le lien : {row['CV']}")
        # Vous pouvez également ajouter ici la logique pour ouvrir le lien dans un nouvel onglet / fenêtre du navigateur.
    st.write("---")  # Ligne de séparation entre chaque entrée


def convert_df(df):
    return df.to_csv(index=False).encode('utf-8')

csv = convert_df(filtered_df_table)

st.download_button(
   "Télécharger les données du tableau",
   csv,
   "file.csv",
   "text/csv",
   key='download-csv'
)


    if st.button("Télécharger les CVs",type="primary", use_container_width=True):
        # Créer un dossier temporaire pour stocker les fichiers CV
        temp_folder = "temp_cvs"
        os.makedirs(temp_folder, exist_ok=True)

        # Télécharger les CV correspondant aux données filtrées
        for index, row in filtered_df.iterrows():
            cv_link = row['CV']
            if pd.notnull(cv_link):
                cv_filename = f"{row['Nom']}_{row['Prenoms']}_CV.pdf"
                cv_path = os.path.join(temp_folder, cv_filename)

                # Télécharger le fichier CV
                with urllib.request.urlopen(cv_link) as response, open(cv_path, 'wb') as out_file:
                    shutil.copyfileobj(response, out_file)

        # Créer le dossier ZIP
        zip_filename = "cvs_filtres.zip"
        zip_path = os.path.join(temp_folder, zip_filename)
        with zipfile.ZipFile(zip_path, 'w') as zip_file:
            for cv_filename in os.listdir(temp_folder):
                cv_path = os.path.join(temp_folder, cv_filename)
                zip_file.write(cv_path, cv_filename)

        # Créer le lien de téléchargement pour le dossier ZIP
        zip_b64 = base64.b64encode(open(zip_path, 'rb').read()).decode()
        href_zip = f'<a href="data:application/zip;base64,{zip_b64}" download="{zip_filename}">Télécharger le dossier ZIP des CV</a>'
        st.markdown(href_zip, unsafe_allow_html=True)

        # Supprimer le dossier temporaire
        shutil.rmtree(temp_folder)




st.data_editor(
    df,
    column_config={
        "CV": st.column_config.LinkColumn(
            "Lien vers CV",
            help="The top trending Streamlit apps",
            display_text="Ouvrir le lien"
        )},
    hide_index=True,
)

In [ ]:
## last version :
############################
## Filtrage des données
# df = df[df['Formation'].isin(selected_formation)]
# df = df[df['Etablissement'].isin(selected_etablissement)]
# df = df[df['Niveau'].isin(selected_niveau)]
filtered_df = df
column_order = ["Nom", "Prenoms", "Niveau", "Formation", "Etablissement","Email", "Telephone", "CV"]
filtered_df_table = filtered_df[column_order]


st.sidebar.write(
  """



  """
)

st.sidebar.write(
  """
  Plus d'infos sur

  www.r2ife-inge.com"""
)

st.sidebar.write(
  """Contacts :

  r2ife.reseau@gmail.com

  +33 767389260 (Murielle Yao)

  +225 09200191 (Kemi Ilupeju)"""
)

# ######################### BOUTONS DE TELECHARGEMENT ############################################################
col1,col2,col3 = st.columns(3)
# Bouton pour télécharger les données du tableau
with col2:
    if st.button("Télécharger le tableau",type="primary", use_container_width=True):
        csv_data = filtered_df.to_csv(index=False)
        b64 = base64.b64encode(csv_data.encode()).decode()
        href = f'<a href="data:file/csv;base64,{b64}" download="donnees_filtrees.csv">Télécharger les données filtrées</a>'
        st.markdown(href, unsafe_allow_html=True)


# Bouton pour télécharger le dossier ZIP des CVs
with col3:
    if st.button("Télécharger les CVs",type="primary", use_container_width=True):
        # Créer un dossier temporaire pour stocker les fichiers CV
        temp_folder = "temp_cvs"
        os.makedirs(temp_folder, exist_ok=True)

        # Télécharger les CV correspondant aux données filtrées
        for index, row in filtered_df.iterrows():
            cv_link = row['CV']
            if pd.notnull(cv_link):
                cv_filename = f"{row['Nom']}_{row['Prenoms']}_CV.pdf"
                cv_path = os.path.join(temp_folder, cv_filename)

                # Télécharger le fichier CV
                with urllib.request.urlopen(cv_link) as response, open(cv_path, 'wb') as out_file:
                    shutil.copyfileobj(response, out_file)

        # Créer le dossier ZIP
        zip_filename = "cvs_filtres.zip"
        zip_path = os.path.join(temp_folder, zip_filename)
        with zipfile.ZipFile(zip_path, 'w') as zip_file:
            for cv_filename in os.listdir(temp_folder):
                cv_path = os.path.join(temp_folder, cv_filename)
                zip_file.write(cv_path,  arcname=cv_filename)

        # Créer le lien de téléchargement pour le dossier ZIP
        zip_b64 = base64.b64encode(open(zip_path, 'rb').read()).decode()
        href_zip = f'<a href="data:application/zip;base64,{zip_b64}" download="{zip_filename}">Télécharger le dossier ZIP des CV</a>'
        st.markdown(href_zip, unsafe_allow_html=True)

        # Supprimer le dossier temporaire
        shutil.rmtree(temp_folder)

# ################# TABLEAU ################################################################################

styles = [
    {'selector': 'th', 'props': [('background-color', 'white'), ('color', 'red')]},
    {'selector': 'tbody', 'props': [('color', 'black')]},
    {'selector': 'td', 'props': [('background-color', 'white'),('white-space', 'nowrap')]}
]

st.table(filtered_df_table.style.set_table_styles(styles))
